In [0]:
import dlt
from pyspark.sql import functions as f

@dlt.view(
    name = 'stabe_scd2',
    comment = 'create a view with cdf batch'
)
def stabe_scd2():
    df = (
        spark.readStream
            .format("delta")
            .option("readChangeFeed", "true")
            .option("startingVersion", 1)
            .table("zz_dev_contabil.default.tabela_3")
    )

    return df


In [0]:
dlt.create_target_table(
    name = "gold_scd2",
    comment = 'create a Slow Changing Dimension Type 2 table from cdf',
    schema = """
        sk BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
        id_tabela_2 INT,
        id_tabela_1 INT,
        sobrenome STRING,
        nome STRING,
        __START_AT timestamp,
        __END_AT timestamp 
    """
)

dlt.apply_changes(
    target = "gold_scd2",
    source = "stabe_scd2",
    keys = ["id_tabela_2"],
    sequence_by = f.col("_commit_timestamp"),
    apply_as_deletes = "_change_type = 'delete'",
    except_column_list = ["_commit_version", "_change_type", "_commit_timestamp"],
    stored_as_scd_type = "2"
)